# Практика
Используемые библиотеки

In [19]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
import random


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
! git clone https://github.com/piton1/DS/ -b Lesson_4
### Если не работает загрузка on-line
df = pd.read_csv("DS/Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

Cloning into 'DS'...
remote: Enumerating objects: 320, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 320 (delta 4), reused 0 (delta 0), pack-reused 306
Receiving objects: 100% (320/320), 25.47 MiB | 8.97 MiB/s, done.
Resolving deltas: 100% (133/133), done.


,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
df['result'] = df['Результат анализа'].apply(lambda x: x.replace(',', '.')).apply(lambda x: re.findall(r'\d+\.\d+|\d+', x)).apply(lambda x: float(x[0]) if len(x) == 1 else None).fillna(0)

def checks(value: float, s: str) -> str:
    s = s.replace(',', '.')
    rule = re.findall(r'не более|в пределах', s)[0]
    norm = list(map(float, re.findall(r'\d+\.\d+|\d+', s)))
    return 'ok' if rule == 'не более' and value < max(norm) or rule == 'в пределах' and max(norm) >= value >= min(norm) else 'not ok'

df['check'] = df.apply(lambda x: checks(x['result'], x['Норматив']), axis=1)
df.set_index(keys='Показатель', inplace=True)
df

,Единица измерений,Результат анализа,Норматив,result,check
Показатель,,,,,
pH,единицы pH,8.4,в пределах 6-9,8.400,ok
Запах,баллы,1,не более 2-3,1.000,ok
Цветность,градусы,б/цвета,не более 30,0.000,ok
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,9.200,ok
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",0.420,ok
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0.017,ok
Нитраты (по NO3),мг/дм3,24,не более 45,24.000,ok
Фосфаты (P),мг/дм3,0.36,"не более 3,5",0.360,ok
Хлориды (Cl),мг/дм3,200,не более 350,200.000,ok


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [37]:
box = ['яблоко'] * 4 + ['orange'] * 5
n = 100000

f = pd.DataFrame({"f": list(map(lambda a: random.sample(box, 3), range(n)))})
f["b"] = f.apply(lambda x: x.f == ['orange'] * 3, axis = 1)
f[f.b].shape[0] / n # Результат совпадает с некоторой погрешностью

0.11909

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [54]:
box = ['~stndrt'] * 3 + ['stndrt'] * 7
n = 10000

s = pd.DataFrame({"s": list(map(lambda a: random.sample(box, 2), range(n)))})
s["b"] = s.apply(lambda x: x.s == ['~stndrt', 'stndrt'], axis = 1)
s[s.b].shape[0] / n # Результат совпадает с некоторой погрешностью

0.2335